In [1]:
from tqdm import tqdm

In [2]:
from libs.feature_extraction.vectorizers import BertVectorizer
import numpy as np
from scipy.signal import stft
import tensorflow as tf

bert = BertVectorizer("microsoft/unixcoder-base")
vectors = bert.load_vectors("data/comment_finder/vectors/bert_vectors.npy")

stft_list=[]

for index, vector in tqdm(enumerate(vectors), total=len(vectors)):
    f, t, Zxx_db = stft(vector, fs = 128, nperseg = 128, noverlap = 116, nfft = 128)
    Zxx_db = tf.abs(Zxx_db)
    Zxx_db = Zxx_db[:64,:64]
    stft_list.append(Zxx_db)
    
stft_list = np.concatenate(stft_list)
display(stft_list.shape)
vectors = stft_list.reshape(stft_list.shape[0]//64, 4096)
display(vectors.shape)

2023-02-15 17:47:39.428949: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-15 17:47:39.574897: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.1/lib64:
2023-02-15 17:47:39.574931: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-15 17:47:40.245380: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open share

Loading model(microsoft/unixcoder-base)...


  0%|          | 0/150677 [00:00<?, ?it/s]2023-02-15 17:47:46.002983: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.1/lib64:
2023-02-15 17:47:46.003079: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.1/lib64:
2023-02-15 17:47:46.003145: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.1/lib64:
2023-02-15 17:47:46.004762: W tensorflow/compiler/xla/stream_executor/platform/default/dso_load

(9643328, 64)

(150677, 4096)

In [3]:
import pandas as pd
from nltk.translate import bleu_score
from numpy.linalg import norm
import numpy as np

df = pd.read_csv("./data/comment_finder/all.csv")

chencherry = bleu_score.SmoothingFunction()

def getBleuScore(test_idx, match_idx):
    ref = df.iloc[[test_idx]].to_records()[0][2]
    hyp = df.iloc[[match_idx]].to_records()[0][2]
    bleu_score_val = bleu_score.sentence_bleu([ref], hyp, smoothing_function=chencherry.method1)
    return bleu_score_val


def find_similiar(data_vector, test_id):

    A = data_vector[test_id]

    max_cosine = 0
    max_sim_id=-1
    for i, B in enumerate(data_vector):
        cosine = np.dot(A,B)/(norm(A)*norm(B))
        if i == test_id:
            continue
        if cosine>max_cosine:
            max_sim_id = i
            max_cosine = cosine
    return max_sim_id, max_cosine

In [4]:
test_df = pd.read_csv("./results/bert_cosine_bleu_C1_F5/fold_1.csv")
test_series = test_df.test_idx

sum = 0
toplam = 1000 #len(test_series)
for index, test_idx in tqdm(enumerate(test_series), total=toplam):
    if index > toplam-1:
        break
    max_sim_id, max_cosine = find_similiar(vectors, test_idx)
    bleu_score_val = getBleuScore(test_idx,max_sim_id)
    sum = sum + bleu_score_val
    
avg_bleu_score = sum / toplam
display(avg_bleu_score)

100%|██████████| 1000/1000 [26:13<00:00,  1.57s/it]


0.13768160258768442